# 뉴스 본문 기반 텍스트 분류 실습  
**TF-IDF + RandomForest + 오분류 분석까지**

이 노트북은 자연어처리(NLP)와 LLM 학습을 위한 사전 실험의 일환으로,  
한국어 뉴스 데이터를 활용하여 본문 내용으로 주제를 분류하는 모델을 만들어봅니다.

- 데이터 출처: [BIGKinds](https://www.bigkinds.or.kr)
- 사용 모델: RandomForestClassifier (Scikit-learn)
- 주요 기법: 전처리, TF-IDF 벡터화, 라벨 인코딩, 분류 모델 학습, 오분류 사례 분석


**참고**

빅카인즈 뉴스 3개월 데이터(20,000건) 중 5,000건 추출하여 진행했습니다.

In [1]:
import pandas as pd

In [5]:
df = pd.read_excel("data/bigkinds_data.xlsx")

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
# 무작위 샘플링 (5,000건)
df_sampled = df.sample(n=5000, random_state=42).reset_index(drop=True)
df_sampled

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,1.010010e+07,20250725,스포츠서울,고봉석 기자,"무주군, ‘무주군 농민회 한마음 가족대회’ 성료",지역>전북,지역>충남,지역>강원,NaN,NaN,NaN,"오광석,황인홍","무풍면,무주군의회,전북,성황리,무풍승지마을,무주군,무주","한마음 가족대회,농협,무주군청,농민회,무주군","무주군,무주군,농민회,한마음,가족,대회,성료,무주군,무주군,농민회,한마음,가족,대회...","무주군,무주,농민회,황인홍,무주군의회,오광석,무풍면,한마음,농업인,무주군청,의원들,...",무주군이 ‘무주군 농민회 한마음 가족대회’ 가 성황리에 개최됐다고 밝혔다. ( 캡션...,https://www.sportsseoul.com/news/read/1533386,NaN
1,4.104008e+06,20250725,프레시안,송부성,"""기후위기 대응, 농가와 함께"" 정읍시, 저탄소 농산물 인증 지원 확대",지역>충북,지역>전북,지역>충남,NaN,NaN,NaN,NaN,"정읍시,전북","한국농업기술진흥원,정읍시,GAP","기후,위기,대응,농가,정읍시,확대,저탄소,농산물,인증,지원,전북,정읍시,농업,분야,...","농산물,저탄소,정읍시,농업인,배출량,백화점,특용작물,인증서,식량작물,부여,중소농가,...",전북 정읍시가 농업 분야 탄소중립 실현을 위한 행정 지원을 강화한다.\n\n정읍시는...,NaN,NaN
2,2.100801e+06,20250725,아시아경제,정두환,안산시 '마을공동체 종합지원센터' 개소,지역>충북,지역>경남,지역>충남,NaN,NaN,NaN,NaN,"바리스타,사동,안산시,상록구,경기도,안산시주민자치회","마을공동체 종합지원센터,마을공동체,안산시,안산시장","종합지원센터,안산시,마을,공동체,종합,지원,센터,개소,공동체,통합,지원,체계,마련,...","공동체,안산시,안산,4층,종합지원센터,이민근,바리스타,어르신들,부스,누리집,지원체계...",경기도 안산시의 마을공동체 활동 거점인 '마을공동체 종합지원센터'가 상록구 사동 1...,https://view.asiae.co.kr/article/2025072510482...,NaN
3,2.100201e+06,20250725,머니투데이,최현승|기자,어린이 화가가 그린 '미래의 바다' 바다상상하기 그림 공모전 시상식,지역>부산,문화>전시_공연,지역>울산,NaN,NaN,NaN,"내빈,박성동,박광명,김종해,이승춘,김명진,박광열,조정희,강호병,서정원","국립해양박물관,경기,부산,인천,평택서","삼진어묵,부산해양경찰서장,한국선급,해양박물관,부산,한국해양소년단연맹,부산광역시,한국...","어린이,화가,미래,바다,바다상상,시상식,그림,공모전,바다상상,그림,공모전,시상식,성...","공모전,시상식,부산,바다상상,해양수산부장관상,부산광역시,머니투데이,별빛,인재상,해양...",[머니투데이 최현승 기자] 제13회 '50년후의 바다상상하기 그림공모전 시상식' 성...,http://news.moneytoday.co.kr/view/mtview.php?n...,NaN
4,1.400351e+06,20250725,중도일보,이상문 기자,대전 청년 월세 지원사업 3000명 모집,지역>대전,경제>부동산,경제>취업_창업,NaN,NaN,NaN,NaN,"대전,환산,9월 30일 '대전 청년 전","대전시청사,국토교통부,대전청년포털","대전,청년,월세,지원,사업,모집,28일,내달,모집,지급,최대,개월,대전시청사,사진제...","대전시,지원사업,홈페이지,신혼부부,39세,19세,이자지원사업,60만,신청일,보증금,...",대전시청사. 사진제공은 대전시대전시는 28일 오전 9시부터 내달 14일 오후 6시까...,https://www.joongdo.co.kr/web/view.php?key=202...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2.100311e+06,20250725,서울경제,안경진 의료전문기자,의정갈등 1년 반만에 수련협의체 첫 회의,사회>사회일반,정치>국회_정당,정치>청와대,NaN,NaN,NaN,"유희철,정은경,한성존,김원섭,박중신,한,김동건","중구,서울,대전협","의대,보건복지부,대한전공의협의회,정부,수련환경평가위원회,비대위,전공,플라자호텔,복지...","의정,갈등,수련협의체,회의,금요일,구체,현안,논의,증원,의대,정원,반발,병원,전공의...","협의체,전공의,전공의들,위원장,대전협,복지부,수련협의체,이형훈,플라자호텔,청원,비대...",의대 정원 증원에 반발해 병원을 떠난 전공의들의 수련 환경 개선 등을 논의하는 수련...,http://www.sedaily.com/NewsView/2GVHLA2V6Z,NaN
4996,2.100701e+06,20250725,헤럴드경제,정호원,위기의 신한카드 상반기 순이익 35% 줄었다,경제>유통,경제>자동차,경제>금융_재테크,NaN,NaN,NaN,NaN,"중구,서울","신한카드,삼성카드","위기,신한카드,상반기,순이익,35%,상반기,누적,당기,순이익,경쟁사,대비,순익,감소...","신한카드,순이익,상반기,당기순이익,삼성카드,희망퇴직,경쟁사,대상자,3793억,246...",상반기 누적 당기순이익 2466억 \n경쟁사 대비 순익 감소폭 두드러져 \n\n 서...,https://biz.heraldcorp.com/article/10539970,NaN
4997,1.601101e+06,20250725,전북일보,강현규,"김제지역, 살인적 폭염 온열질환자 급증",사회>의료_건강,NaN,NaN,재해>자연재해>폭염,NaN,NaN,NaN,"이주,영등포구,여의대로,서울,김제지역,여의도,김제","119구급대,소방서장,김제소방서,119구급차,의용소방대,연합뉴스 연일 체감온도","김제지역,살인,폭염,온열,질환자,급증,전국,지역,전국,폭염,특보,발효,여의대로,서울...","온열질환자,온열질환,구급차,질환자,구급대,김제소방서,김제,야외활동,열탈진,이종옥,서...",전국 대부분 지역에 폭염 특보가 발효된 25일 서울 영등포구 여의도 여의대로에 아지...,https://www.jjan.kr/article/20250725580044,NaN
4998,1.101101e+06,20250725,한국일보,NaN,"제이씨지, 경기도의회 AI 챗봇 사업 수주 '사람 중심' AI로 공공 디지털 혁신 ...",IT_과학>IT_과학일반,NaN,NaN,NaN,NaN,NaN,"심재성,김호겸,이정표","서울,코엑스","경기환경에너지진흥원,위원회,현요셉,정보,ISP,세울,경기도의회,탄소중립지원센터,제이...","제이씨,수주,경기도의회,AI,챗봇,사업,사람,중심,AI,공공,디지털,혁신,선도,제이...","ai,챗봇,경기도의회,제이,실시간,제이씨지,경기도,씨지,제이씨,위원회,기후아이,의정...",전문 AI 기술 기업 제이씨지가 경기도의회의 ‘생성형 AI 기반 사무혁신 방안’ 및...,https://www.hankookilbo.com/News/Read/A2025072...,NaN


In [9]:
# 필요한 컬럼만 추출
df_clf = df_sampled[['본문', '통합 분류1']].copy()

# 결측치 제거
df_clf = df_clf.dropna(subset=['본문', '통합 분류1'])

# 결과 확인
df_clf.head()

,본문,통합 분류1
0,무주군이 ‘무주군 농민회 한마음 가족대회’ 가 성황리에 개최됐다고 밝혔다. ( 캡션...,지역>전북
1,전북 정읍시가 농업 분야 탄소중립 실현을 위한 행정 지원을 강화한다.\n\n정읍시는...,지역>충북
2,경기도 안산시의 마을공동체 활동 거점인 '마을공동체 종합지원센터'가 상록구 사동 1...,지역>충북
3,[머니투데이 최현승 기자] 제13회 '50년후의 바다상상하기 그림공모전 시상식' 성...,지역>부산
4,대전시청사. 사진제공은 대전시대전시는 28일 오전 9시부터 내달 14일 오후 6시까...,지역>대전


In [10]:
from sklearn.preprocessing import LabelEncoder

# 인코더 생성 및 라벨 정수 변환
le = LabelEncoder()
df_clf['label'] = le.fit_transform(df_clf['통합 분류1'])

# 라벨별 개수 확인
df_clf['label'].value_counts()


,count
label,
15,235
61,196
45,190
81,179
66,173
...,...
53,7
7,6
55,6


## 텍스트 전처리 및 벡터화

- 뉴스 본문에서 특수문자, 괄호, 줄바꿈 등을 제거합니다  .
- 불용어 리스트를 지정하여 불필요한 단어를 제거합니다  .
- TF-IDF 방식으로 텍스트를 벡터화하여 모델에 입력할 수 있도록 변환합니다.

In [19]:
import re

# 기본적인 특수문자 제거 함수
def clean_text(text):
    text = re.sub(r'\n', ' ', text)                      # 줄바꿈 제거
    text = re.sub(r'\[[^\]]*\]', '', text)               # 대괄호 안 내용 제거 [기자], [사진] 등
    text = re.sub(r'\([^\)]*\)', '', text)               # 괄호 안 내용 제거 (캡션), (사진제공)
    text = re.sub(r'[^ㄱ-ㅎ가-힣a-zA-Z0-9\s]', ' ', text) # 한글, 영문, 숫자, 공백 제외 제거
    text = re.sub(r'\s+', ' ', text)                     # 중복 공백 제거
    return text.strip()

# 뉴스 본문 전처리
df_clf['clean_text'] = df_clf['본문'].apply(clean_text)
df_clf.head()


,본문,통합 분류1,label,clean_text
0,무주군이 ‘무주군 농민회 한마음 가족대회’ 가 성황리에 개최됐다고 밝혔다. ( 캡션...,지역>전북,78,무주군이 무주군 농민회 한마음 가족대회 가 성황리에 개최됐다고 밝혔다 전북 무주군은...
1,전북 정읍시가 농업 분야 탄소중립 실현을 위한 행정 지원을 강화한다.\n\n정읍시는...,지역>충북,82,전북 정읍시가 농업 분야 탄소중립 실현을 위한 행정 지원을 강화한다 정읍시는 오는 ...
2,경기도 안산시의 마을공동체 활동 거점인 '마을공동체 종합지원센터'가 상록구 사동 1...,지역>충북,82,경기도 안산시의 마을공동체 활동 거점인 마을공동체 종합지원센터 가 상록구 사동 15...
3,[머니투데이 최현승 기자] 제13회 '50년후의 바다상상하기 그림공모전 시상식' 성...,지역>부산,75,제13회 50년후의 바다상상하기 그림공모전 시상식 성료 상상력과 창의성의 결실 50...
4,대전시청사. 사진제공은 대전시대전시는 28일 오전 9시부터 내달 14일 오후 6시까...,지역>대전,74,대전시청사 사진제공은 대전시대전시는 28일 오전 9시부터 내달 14일 오후 6시까지...


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 불용어 리스트 (간단한 예시)
stopwords_kr = ['기자', '사진', '제공', '밝혔다', '있다', '위해', '것', '등', '에서', '으로', '하고', '하는', '했다', '된다', '하며', '한', '가', '은', '는', '을', '를']

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=10000, stop_words=stopwords_kr)
X = vectorizer.fit_transform(df_clf['본문'])
y = df_clf['label']


## 분류 모델 학습 및 평가

- TF-IDF 벡터를 입력으로 사용하고, 라벨은 `통합 분류1`의 인코딩 값을 사용합니다 .
- 학습 데이터와 테스트 데이터를 8:2 비율로 분리하여 모델을 학습하고, 검증 정확도를 확인합니다.

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. 학습/검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y,                 # 입력(X), 정답(y)
    test_size=0.2,        # 검증용 데이터 비율: 전체의 20%
    random_state=42,      # 결과 재현성을 위한 랜덤 시드
    stratify=y            # 클래스 비율을 유지하면서 나누기
)


# 2. 모델 정의 및 학습
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 3. 예측 및 평가
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("검증 정확도: {:.2f}%".format(accuracy * 100))


전처리 적용 후 검증 정확도: 43.10%


## 오분류 사례 분석

모델이 예측을 틀린 뉴스 기사들을 직접 확인하고,  
왜 틀렸는지를 텍스트 기반으로 해석해보는 과정입니다.  
이를 통해 모델의 한계나 라벨 분류 기준의 애매함 등을 파악할 수 있습니다.

In [23]:
# 1. 예측 vs 실제 라벨 비교
misclassified_idx = (y_test != y_pred)  # 오분류된 인덱스 불리언 배열

# 2. 원래 라벨 이름으로 복원
actual_labels = le.inverse_transform(y_test[misclassified_idx])
predicted_labels = le.inverse_transform(y_pred[misclassified_idx])

# 3. 오분류된 본문 원문 가져오기
misclassified_texts = df_clf.iloc[y_test.index[misclassified_idx]]['본문'].values

# 4. 상위 5개 출력
for i in range(5):
    print(f"[실제 라벨] {actual_labels[i]}")
    print(f"[예측 라벨] {predicted_labels[i]}")
    print(f"[본문 요약] {misclassified_texts[i][:200]}...")
    print("-" * 80)


[실제 라벨] 사회>여성
[예측 라벨] 사회>의료_건강
[본문 요약] 여성 탈모가 환자의 삶의 질은 물론 우울감과 불안감에도 깊은 영향을 미친다는 연구 결과가 나왔다. 특히 탈모가 심할수록 이러한 심리적 부담은 더 커졌다. 
 
25일 인하대병원에 따르면 병원 의생명연구원장인 최광성 피부과 교수는 최근 '여성형 탈모증 환자의 삶의 질 및 심리사회적 영향에 대한 연구'라는 논문에서 13세 이상 여성 환자 202명을 대상으로.....
--------------------------------------------------------------------------------
[실제 라벨] 경제>증권_증시
[예측 라벨] 경제>유통
[본문 요약] [그래픽=아주경제] 
상장지수펀드(ETF)가 국내 펀드 시장 주도권을 확고히 다졌다. 높은 수익률과 함께 다양한 테마를 선보이며 투자자의 선택지를 넓히고 있다. 
  
25일 한국예탁결제원에 따르면 최근 1년 기준 펀드 수익률 상위 10개 펀드 중 9개가 ETF인 것으로 나타났다. 특히 '한화 PLUS K방산 ETF'(198.08%)를 비롯해 'TIGE.....
--------------------------------------------------------------------------------
[실제 라벨] 지역>경기
[예측 라벨] 지역>대전
[본문 요약] 광주시가 전기요금 절감과 온실가스 감축 실현을 위한 '탄소중립형 승강기' 사업을 시행한다. 
 
광주시는 시의회, 한국전력공사 광주전남본부, 한국승강기안전공단 호남지역본부와 함께 25일 시의회 대회의실에서 '승강기 회생제동장치 설치 지원사업 추진을 위한 업무협약'을 체결했다. 
 
이 사업은 탄소중립과 기후위기 대응을 위해 광주시가 올해 처음 도입한 것.....
--------------------------------------------------------------------------------
[실제 라벨] 지역>대전
[예측 라벨] 지역>충남
[본문

- TF-IDF 기반 벡터화 + RandomForestClassifier로 뉴스 본문 분류 실습 완료  
- 전처리 방식에 따라 모델 성능이 크게 달라짐을 확인  
- 오분류 분석을 통해 분류 기준의 모호성, 지역명 혼동, 키워드 부족 등의 문제 파악  
- 이후에는 형태소 분석, 키워드 결합, 라벨 단순화 등을 통해 성능 향상 시도 예정